In [1]:
from tqdm import tqdm
import pandas as pd
import datetime as dt
import numpy as np
import warnings
import pymssql
import locale
import glob
import os

warnings.filterwarnings('ignore')
pd.set_option('use_inf_as_na', True)
pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)
locale.setlocale(locale.LC_ALL, 'ru_RU.UTF-8')

'ru_RU.UTF-8'

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Название склада], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Название товара], [Остаток], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [ШК], [Код товара]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'СИ', N'ИФ', N'ПДК', N'ДК', N'ВЫРИЦА', N'ЧАСЫ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [4]:
stock = stock[stock['ID Сайта'] != '']

In [5]:
stock['Код склада'] = stock['Код склада'].astype(int)

In [6]:
year_now, month_now = str(dt.datetime.today().year), str(dt.datetime.today().month)
if int(month_now) < 10: month_now = '0' + str(month_now)

In [7]:
os.chdir(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new')
extension = 'xlsx'
filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames = list()

for file in filenames:
    if year_now in file and month_now in file: 
        all_filenames.append(file) 

In [8]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [05:55<00:00, 71.08s/it]


In [9]:
sale.columns = ['КБК', 'Дата', 'Продавец', 'Номер чека', 'ID продажи', 'Тов',
    'article', 'Количество', 'Номенклатура', 'Номер карты',
    'Товарная группа', 'Товарное направление', 'Тип 1',
    'Списание СберСпасибо', 'Группа наценки', 'Вес', 'Проба', 'Размер',
    'ШК', 'Цена без скидки', 'Базовая скидка', 'Скидка в руб.', 'Название',
    'Цена после скидки', 'Phone', 'Себестоимость', 'Сумма', 'Услуга',
    'Общая сумма']

In [10]:
articles = stock['Артикул поставщика'].tolist()

In [11]:
sale = sale.query('article in @articles')
sale = sale.groupby(by = 'article', as_index=False)[['Количество', 'Общая сумма']].sum()
sale.columns = ['Артикул поставщика', 'Количество продаж за предыдущую неделю', 'Общая сумма продаж за предыдущю неделю']

In [12]:
center = stock[stock['Код склада'] == 3075]
center = center.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
center.columns = ['ID Сайта', 'Остаток на центральном складе']

In [13]:
kolambus = stock[stock['Код склада'] == 5442]
kolambus = kolambus.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
kolambus.columns = ['ID Сайта', 'Остаток на складе коламбуса']

In [14]:
federal = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Федеральные склады 11.06.xlsx') # подтянуть более актуальный файл
stock.rename(columns={'Название склада': 'name'}, inplace=True)
federal = federal['outlet_name'].tolist() 
federal = stock.query('name in @federal')
federal = federal.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()
federal.columns = ['ID Сайта', 'Остаток на федеральных складах']

In [15]:
stock = stock.merge(federal, how='outer', on='ID Сайта')
stock = stock.merge(kolambus, how='outer', on='ID Сайта')
stock = stock.merge(center, how='outer', on='ID Сайта')

In [16]:
conn = pymssql.connect(server='spb-s-sql-dwh', database='DWH_DEV', charset='WINDOWS-1251')
cursor = conn.cursor()

sql = f'''
SELECT [article], [sort]
from [DWH_DEV].[ecom].[catalog_product]
'''

sql_query = pd.read_sql_query(sql, conn)
sort = pd.DataFrame(sql_query)

In [17]:
sort.rename(columns={'article': 'ID Сайта'}, inplace=True)
stock = stock.merge(sort, how='outer', on='ID Сайта')
stock = stock.merge(sale, how='outer', on='Артикул поставщика')

In [18]:
summ = stock.groupby(by='ID Сайта', as_index=False)['Остаток'].sum()
summ.rename(columns={'Остаток': 'Общий остаток на всех складах'}, inplace=True)
stock = stock.merge(summ, how='left', on='ID Сайта')
stock = stock.drop_duplicates(subset=['ID Сайта', 'Общий остаток на всех складах'])

In [19]:
stock['Остаток на складе коламбуса'].fillna(0, inplace=True)
stock['Остаток на федеральных складах'].fillna(0, inplace=True)
stock['Остаток на центральном складе'].fillna(0, inplace=True)

In [20]:
stock['Сумма в себестоимости на остатках ОБЩ'] = stock['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] * stock['Общий остаток на всех складах']
stock['Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР'] = (stock['Остаток на складе коламбуса'] + stock['Остаток на федеральных складах'] + stock['Остаток на центральном складе']) * stock['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [21]:
stock.dropna(subset=['name'], inplace=True)

In [22]:
conn = pymssql.connect(server='spb-s-sql-dwh', database='DWH_DEV', charset='WINDOWS-1251')
cursor = conn.cursor()

productimage = f'''
select 
[image], [sort], [product_id], [deleted]
from [DWH_DEV].[ecom].[catalog_productimage]
where [deleted] is NULL
'''

product = f'''
select 
[id], [article], [published]
from [DWH_DEV].[ecom].[catalog_product]
'''

catalog_productimage = pd.read_sql_query(productimage, conn)
catalog_product = pd.read_sql_query(product, conn)

In [23]:
catalog_product = catalog_product.query('published == True')
catalog_productimage.rename(columns={'product_id': 'id'},
                            inplace=True)
images = catalog_product.merge(catalog_productimage, how='left', on='id')
images['image'] = 'https://static2.585.cloud/media/' + images['image']
images = images.query('sort == 1')
images = images[['article', 'image', 'published']]
# date = dt.date.today().strftime('%d.%m.%Y')
# images.to_excel(f'Актуальные фото с сайта {date}.xlsx', index=False)

In [24]:
stock['ID Сайта'] = stock['ID Сайта'].astype(str)

In [25]:
images.rename(columns={'article': 'ID Сайта'}, inplace=True)
images['ID Сайта'] = images['ID Сайта'].astype(str)

In [26]:
stock = stock.merge(images, how='outer', on='ID Сайта')
stock = stock.drop(columns=['Остаток', 'Код склада'])

In [27]:
stock.dropna(subset=['name'], inplace=True)

In [28]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime) # мы ищем тут последний выгруженный файл (занимается выгрузкой Оризу Наталья email: Orizu.Natalya@zolotoy.ru)

ecom = pd.read_excel(latest_file)

path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [29]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1','discount_value_promotions_2', 'discount_value_promotions_3','discount_value_promotions_4']
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [30]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1
    
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [31]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'Цена со скидками Ecom']]
ecom.columns = ['ID Сайта', 'Цена со скидками Ecom']
stock = stock.merge(ecom, how='left', on='ID Сайта')
stock = stock.drop_duplicates(subset=['ID Сайта'])

In [32]:
stock = stock.drop(columns=['name', 'Код объекта', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)'])

In [33]:
stock.columns = ['Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Название товара', 'ШК', 'Код товара', 'Остаток на федеральных складах',
    'Остаток на складе коламбуса', 'Остаток на центральном складе', 'sort',
    'Количество продаж за предыдущую неделю',
    'Общая сумма продаж за предыдущю неделю',
    'Общий остаток на всех складах',
    'Сумма в себестоимости на остатках ОБЩ',
    'Сумма в себестоимости на остатках ФЕД+КОЛАМБУС+ЦЕНТР', 'Ссылка на изображение', 
    'Опубликован', 'Цена со скидками Ecom']

In [34]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')['Артикул сайта'].astype(str).tolist() # сюда более актуальный файл

stock['Запрет к размещению'] = np.nan
for i in stock.index:
    if stock['ID Сайта'][i] in zapret: stock['Запрет к размещению'][i] = True
    else: stock['Запрет к размещению'][i] = False

In [35]:
stock['Запрет к размещению'].value_counts()

Запрет к размещению
False    94215
True        44
Name: count, dtype: int64

In [36]:
stock.dropna(subset=['ШК'], inplace=True)

In [37]:
stock1 = stock[~stock['Ссылка на изображение'].astype(str).str.contains('http')]
stock2 = stock[(stock['Ссылка на изображение'].astype(str).str.contains('http')) & (stock['Запрет к размещению'] == False)]

In [38]:
stock = pd.concat([stock1, stock2], ignore_index=True)

In [39]:
stock = stock[(stock['Остаток на складе коламбуса'] != 0) | (stock['Остаток на федеральных складах'] != 0) | (stock['Остаток на центральном складе'] != 0)]

In [40]:
stock.to_excel(fr'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Отчётность для контента\\{dt.datetime.today().strftime('%d.%m.%Y')} для контента.xlsx', index=False)